In [4]:
from testdataextractor.testdataextractor.extractor import Extractor
from summpy.summpy import lexrank
import pandas as pd

## Get some data

In [139]:

testarticles = [1957284403,1965754064,233465322,283147769,362778020,37793736,389321649,540607195,60134403,887344770, ]
all_articles = []
all_sets_sentences = []
all_article_sentences = []
all_comments_sentences = []
all_groups = [] # each comment is a group, the article counts as a comment
for art in testarticles:
    ext = Extractor("../test_data/{0}.ofs.gold.xml".format(art))
    article = ext.extract(verbose=True)
    all_articles.append(article)
    
    df_article = pd.DataFrame.from_dict(article['sentences'], orient='index')
    sorted_indexes = [ "s{0}".format(x) for x in range(len(article['sentences'].values()))]
    sentences = list(df_article.ix[sorted_indexes, 'text'])
    
    ordered_sentences = df_article.ix[sorted_indexes]
    article_sentences_ix = ordered_sentences.ix[:,'comment'].isnull()
    
    art_sentences = ordered_sentences[article_sentences_ix]
    com_sentences = ordered_sentences[article_sentences_ix == False]
    
    article_sentences = list(art_sentences['text'])
    comment_sentences = list(com_sentences['text'])
    
    groups_of_sentences = ordered_sentences.groupby(by='comment', sort=False)
    groupcount = len(groups_of_sentences.groups.keys())
    index = ["c{0}".format(i) for i in range(groupcount)]
    grouped_comments = []
    for g in index:
        com = " | ".join(list(groups_of_sentences.get_group(g)['text']))
        grouped_comments.append(com)

    grouped_comments.append(" | ".join(article_sentences))
    
    all_article_sentences.append(article_sentences)
    all_comments_sentences.append(comment_sentences)
    all_groups.append(grouped_comments)
    
    if df_article.ix['s2', 'text'] == sentences[2]:
        print "Extracted list of sentences is in a proper order."
        all_sets_sentences.append(sentences)
    else:
        print "Extracted list of sentences is unordered."


50  comments parsed.
190  sentences parsed.
140  links parsed.
Extracted list of sentences is in a proper order.
50  comments parsed.
203  sentences parsed.
102  links parsed.
Extracted list of sentences is in a proper order.
50  comments parsed.
290  sentences parsed.
199  links parsed.
Extracted list of sentences is in a proper order.
49  comments parsed.
178  sentences parsed.
92  links parsed.
Extracted list of sentences is in a proper order.
50  comments parsed.
170  sentences parsed.
79  links parsed.
Extracted list of sentences is in a proper order.
50  comments parsed.
184  sentences parsed.
1  links parsed.
Extracted list of sentences is in a proper order.
49  comments parsed.
278  sentences parsed.
25  links parsed.
Extracted list of sentences is in a proper order.
49  comments parsed.
156  sentences parsed.
26  links parsed.
Extracted list of sentences is in a proper order.
50  comments parsed.
196  sentences parsed.
14  links parsed.
Extracted list of sentences is in a prop

## Feed data into lexrank
The summpy MIT licensed repo used for this task, returns a tuple with a dictionary with sentences index + score, and the similarity_matrix.

In [140]:
all_ranked_sentences = []
all_matrxs = []
all_ranked_art_sentences = []
all_art_matrxs = []
all_ranked_com_sentences = []
all_com_matrxs = []

def lexrank_all_lists_of_sentences(sentences_lists):
    all_ranked = []
    all_matrixes = []
    for sentences in sentences_lists:
        ranked, matrix = lexrank.lexrank(sentences)
        all_ranked_sentences.append(ranked)
        all_matrixes.append(matrix)
    return all_ranked, all_matrixes

all_ranked_sentences, all_matrxs = lexrank_all_lists_of_sentences(
    all_sets_sentences
)

all_ranked_art_sentences, all_art_matrxs = lexrank_all_lists_of_sentences(
    all_article_sentences
)

all_ranked_com_sentences, all_com_matrxs = lexrank_all_lists_of_sentences(
    all_comments_sentences
)


## See the data

In [141]:
all_ranked_dfs = []
all_similarity_dfs = []
for ranked_sentences, similarity_mtrx in zip(all_ranked_sentences, all_matrxs):
    df_ranked_sentences = pd.DataFrame.from_dict(ranked_sentences,
                                                 orient='index')
    df_similarity_mtrx = pd.DataFrame(similarity_mtrx)
    all_ranked_dfs.append(df_ranked_sentences)
    all_similarity_dfs.append(df_similarity_mtrx)

all_similarity_dfs[0]

,0,1,2,3,4,5,6,7,8,9,...,180,181,182,183,184,185,186,187,188,189
0,1.000000,0.295122,0.263523,0.081650,0.188562,0.248069,0.000000,0.166091,0.072548,0.193649,...,0.048795,0.083771,0.149071,0.117670,0.121466,0.000000,0.077267,0.000000,0.000000,0.111803
1,0.295122,1.000000,0.397498,0.428384,0.463739,0.142354,0.294514,0.476554,0.095157,0.275167,...,0.176006,0.480717,0.268853,0.231512,0.398304,0.039193,0.164689,0.034565,0.169334,0.366618
2,0.263523,0.397498,1.000000,0.215166,0.273297,0.228802,0.215166,0.372035,0.114708,0.136083,...,0.154303,0.309058,0.196419,0.186052,0.341432,0.062994,0.203616,0.055556,0.068041,0.206239
3,0.081650,0.428384,0.215166,1.000000,0.384900,0.202548,0.066667,0.372935,0.177705,0.263523,...,0.199205,0.205196,0.304290,0.288231,0.231413,0.097590,0.189264,0.086066,0.052705,0.182574
4,0.188562,0.463739,0.273297,0.384900,1.000000,0.233882,0.192450,0.371906,0.102598,0.182574,...,0.230022,0.335664,0.281091,0.258860,0.314929,0.056344,0.218543,0.099381,0.091287,0.237171
5,0.248069,0.142354,0.228802,0.202548,0.233882,1.000000,0.101274,0.180259,0.134976,0.120096,...,0.121046,0.155857,0.184900,0.194602,0.200881,0.074125,0.167716,0.130744,0.000000,0.138675
6,0.000000,0.294514,0.215166,0.066667,0.192450,0.101274,1.000000,0.135613,0.118470,0.105409,...,0.039841,0.341993,0.060858,0.032026,0.214883,0.097590,0.031544,0.086066,0.105409,0.228218
7,0.166091,0.476554,0.372035,0.372935,0.371906,0.180259,0.135613,1.000000,0.120495,0.321634,...,0.303915,0.278271,0.278543,0.293158,0.353053,0.049629,0.224583,0.043769,0.080408,0.208907
8,0.072548,0.095157,0.114708,0.177705,0.102598,0.134976,0.118470,0.120495,1.000000,0.140488,...,0.070799,0.091161,0.108148,0.170733,0.102808,0.086711,0.224221,0.076472,0.093659,0.040555
9,0.193649,0.275167,0.136083,0.263523,0.182574,0.120096,0.105409,0.321634,0.140488,1.000000,...,0.157485,0.108148,0.096225,0.101274,0.065339,0.077152,0.074813,0.068041,0.041667,0.108253


# Cluster Similarity Matrixes DBSCAN
It requires the matrix to be normalized between 0 to 1, but the cosine similarity is already that. Also, the similarity matrix needs to have distances, which is done by doing 1 - pairwise_similarity (this is already taken care off by the lexrank algorithm)

In [142]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics

def print_results(db):
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    print('Estimated number of clusters: %d' % n_clusters_)

for i, (article, comments) in enumerate(zip(all_art_matrxs, all_com_matrxs)):    
    db_a = DBSCAN(metric='precomputed', eps=0.3, min_samples=2).fit(article)
    db_c = DBSCAN(metric='precomputed', eps=0.3, min_samples=2).fit(comments)
    
    print "\nARTICLE {0}".format(i)
    print_results(db_a)
    print "\nCOMMENTS {0}".format(i)
    print_results(db_c)


ARTICLE 0
Estimated number of clusters: 1

COMMENTS 0
Estimated number of clusters: 1

ARTICLE 1
Estimated number of clusters: 1

COMMENTS 1
Estimated number of clusters: 1

ARTICLE 2
Estimated number of clusters: 1

COMMENTS 2
Estimated number of clusters: 1

ARTICLE 3
Estimated number of clusters: 1

COMMENTS 3
Estimated number of clusters: 1

ARTICLE 4
Estimated number of clusters: 1

COMMENTS 4
Estimated number of clusters: 1

ARTICLE 5
Estimated number of clusters: 1

COMMENTS 5
Estimated number of clusters: 1

ARTICLE 6
Estimated number of clusters: 1

COMMENTS 6
Estimated number of clusters: 1

ARTICLE 7
Estimated number of clusters: 1

COMMENTS 7
Estimated number of clusters: 1

ARTICLE 8
Estimated number of clusters: 1

COMMENTS 8
Estimated number of clusters: 1

ARTICLE 9
Estimated number of clusters: 1

COMMENTS 9
Estimated number of clusters: 1


# Vectorize sentences just to check if it is better if I do this myself other than lexrank

In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem

english_stemmer = nltk.stem.SnowballStemmer('english')

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer=super(StemmedTfidfVectorizer,self).build_analyzer()
        return lambda doc:(english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=1, stop_words='english')



In [144]:
article_s =  all_article_sentences
comment_s = all_comments_sentences
for i, (article, comments) in enumerate(zip(article_s, comment_s)): 
    X_article = vectorizer.fit_transform(article)
    X_comments = vectorizer.fit_transform(comments)
    db_a = DBSCAN(eps=0.3, min_samples=2).fit(X_article)
    db_c = DBSCAN(eps=0.3, min_samples=2).fit(X_comments)
    
    print "\nARTICLE {0}".format(i)
    print_results(db_a)
    print "\nCOMMENTS {0}".format(i)
    print_results(db_c)


ARTICLE 0
Estimated number of clusters: 0

COMMENTS 0
Estimated number of clusters: 1

ARTICLE 1
Estimated number of clusters: 0

COMMENTS 1
Estimated number of clusters: 3

ARTICLE 2
Estimated number of clusters: 0

COMMENTS 2
Estimated number of clusters: 4

ARTICLE 3
Estimated number of clusters: 0

COMMENTS 3
Estimated number of clusters: 6

ARTICLE 4
Estimated number of clusters: 0

COMMENTS 4
Estimated number of clusters: 3

ARTICLE 5
Estimated number of clusters: 0

COMMENTS 5
Estimated number of clusters: 1

ARTICLE 6
Estimated number of clusters: 0

COMMENTS 6
Estimated number of clusters: 2

ARTICLE 7
Estimated number of clusters: 0

COMMENTS 7
Estimated number of clusters: 2

ARTICLE 8
Estimated number of clusters: 0

COMMENTS 8
Estimated number of clusters: 3

ARTICLE 9
Estimated number of clusters: 0

COMMENTS 9
Estimated number of clusters: 1


# Don't cluster sentences, cluster comments
Count the *article* as a comment.

In [145]:
for i, group in enumerate(all_groups): 
    X_group = vectorizer.fit_transform(group)

    db_a = DBSCAN(eps=1, min_samples=2).fit(X_group)
    
    print "\nArticle {0}'s groups".format(i)
    print_results(db_a)


Article 0's groups
Estimated number of clusters: 0

Article 1's groups
Estimated number of clusters: 2

Article 2's groups
Estimated number of clusters: 0

Article 3's groups
Estimated number of clusters: 3

Article 4's groups
Estimated number of clusters: 1

Article 5's groups
Estimated number of clusters: 0

Article 6's groups
Estimated number of clusters: 1

Article 7's groups
Estimated number of clusters: 2

Article 8's groups
Estimated number of clusters: 2

Article 9's groups
Estimated number of clusters: 1


# Skip the clustering for grouping, use comments and the article as groups

In [195]:
from gensim import corpora, models, similarities
from nltk.tokenize import TweetTokenizer
import nltk.stem

def preprocess_docs(documents):
    tokenizer = TweetTokenizer()
    english_stemmer = nltk.stem.SnowballStemmer('english')

    texts = [tokenizer.tokenize(d) for d in documents]

    stemmed_texts = []
    for text in texts:
        stemmed_text = [english_stemmer.stem(t) for t in text]
        stemmed_texts.append(stemmed_text)
    return stemmed_texts

def query_all(lsi_queries, index):
    total_pairs = 0
    selected_pairs = 0
    strong_sentences = 0
    similarity_dict = {}
    for i, query in enumerate(lsi_queries):
        sims = index[query]
        strong_sims = [s for s in list(enumerate(sims)) if s[1] > 0.99]
        pairs = len(strong_sims)
        if pairs > 1 and pairs < 8:
#             print "\ns{0} similarities with comments".format(i+comment_start_index)
#             print "{0} strong similarity pairs".format(len(strong_sims))
#             print all_comments_sentences[0][i]
            strong_sentences += 1
            selected_pairs += pairs
            similarity_dict[i] = strong_sims

        total_pairs += pairs
    print "\n{0} strong sentences".format(strong_sentences)
    print "{0} total sentence-group pairs".format(selected_pairs)
    print total_pairs/float(len(lsi_queries)) 
    return similarity_dict

def find_links_between_in(documents, comments_sentences,
                          article_sentences):    
    stemmed_texts = preprocess_docs(documents)
    dictionary = corpora.Dictionary(stemmed_texts)
    dictionary.filter_extremes(no_below=1, no_above=0.8)
    corpus = [dictionary.doc2bow(text) for text in stemmed_texts]

    lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
    index = similarities.MatrixSimilarity(lsi[corpus])

#     comment_start_index = len(article_sentences)
    stemmed_queries = preprocess_docs(comments_sentences)
    query_dict = corpora.Dictionary(stemmed_queries)
    lsi_queries = [lsi[query_dict.doc2bow(text)] for text in stemmed_queries]
    similarity_dict = query_all(lsi_queries, index)
    return similarity_dict

documents = all_groups[0]
comments_s = all_comments_sentences[0]
article_s = all_article_sentences[0]

def find_links_in_all_articles():
    for i, docs in enumerate(all_groups):
        comments = all_comments_sentences[i]
        article = all_article_sentences[i]
        print "\n\nARTICLE {0}".format(i)
        find_links_between_in(docs, comments, article)


% time find_links_in_all_articles()



ARTICLE 0

33 strong sentences


174 total sentence-group pairs
7.70676691729


ARTICLE 1

32 strong sentences


134 total sentence-group pairs
8.54347826087


ARTICLE 2

54 strong sentences


163 total sentence-group pairs
10.0778688525


ARTICLE 3

45 strong sentences


177 total sentence-group pairs
6.13432835821


ARTICLE 4

44 strong sentences


173 total sentence-group pairs
3.85810810811


ARTICLE 5

40 strong sentences


194 total sentence-group pairs
6.44755244755


ARTICLE 6

31 strong sentences


134 total sentence-group pairs
7.16666666667


ARTICLE 7

53 strong sentences


243 total sentence-group pairs
6.60330578512


ARTICLE 8

40 strong sentences


158 total sentence-group pairs
7.4965034965


ARTICLE 9

64 strong sentences
322 total sentence-group pairs
6.29133858268
CPU times: user 2.55 s, sys: 109 ms, total: 2.66 s
Wall time: 2.51 s


# Sentence-sentence pairs with gensim

In [196]:
def find_sentence_links_in_all_articles():
    all_similarity_dicts = []
    for i, docs in enumerate(all_sets_sentences):
        comments = all_comments_sentences[i]
        article = all_article_sentences[i]
        print "\n\nARTICLE {0}".format(i)
        s_dict = find_links_between_in(docs, comments, article)
        all_similarity_dicts.append(s_dict)
    return all_similarity_dicts

% time all_similarity_dicts = find_sentence_links_in_all_articles()



ARTICLE 0

11 strong sentences


49 total sentence-group pairs
42.7894736842


ARTICLE 1

3 strong sentences


15 total sentence-group pairs
39.9130434783


ARTICLE 2

8 strong sentences


24 total sentence-group pairs
43.5573770492


ARTICLE 3

10 strong sentences


36 total sentence-group pairs
26.7611940299


ARTICLE 4

45 strong sentences


273 total sentence-group pairs
20.0067567568


ARTICLE 5

4 strong sentences


24 total sentence-group pairs
28.0979020979


ARTICLE 6

2 strong sentences


14 total sentence-group pairs
50.7264957265


ARTICLE 7

7 strong sentences


35 total sentence-group pairs
30.7851239669


ARTICLE 8

5 strong sentences


18 total sentence-group pairs
29.1188811189


ARTICLE 9

1 strong sentences
4 total sentence-group pairs
28.842519685
CPU times: user 3.12 s, sys: 132 ms, total: 3.26 s
Wall time: 3.08 s


In [211]:
def output_top_sentence_pairs(s_dict, all_art_sentences,
                              all_sentences, 
                              all_comment_sentences):
    d = s_dict
    comment_start_index = len(all_art_sentences)
    sentences = all_sentences
    comment_sentences = all_comment_sentences
    
    for comment_sentence, links in zip(d.keys(),d.values()):

        s1_id = "s{0}".format(comment_sentence + comment_start_index)
        s2_id = "s{0}".format(links[0][0])
        print "\nLink found"
        print comment_sentence+comment_start_index, [l[0] for l in links]        
        print comment_sentences[comment_sentence]
        for l_id, prob in links:
            print sentences[l_id]

for i, _ in enumerate(all_articles):
    print "\nARTICLE {0}==========".format(i)
    output_top_sentence_pairs(all_similarity_dicts[i],
                              all_article_sentences[i],
                              all_sets_sentences[i],
                              all_comments_sentences[i])


ARTICLE 0==========

Link found
185 [59, 68, 95, 117]
.org/wiki/ 
[alex442] Human Rights ? 
Haw haw ! 
Resist ! 
Freedom ? 

Link found
91 [10, 11, 34]
Quite so . 
It gives top secret codenames for each firm , with BT ( " Remedy " ) , Verizon Business ( " Dacron " ) , and Vodafone Cable ( " Gerontic" ) . 
The other firms include Global Crossing ( " Pinnage " ) , Level 3 ( " Little " ) , Viatel ( "Vitreous " ) and Interoute ( " Streetcar " ) . 
The source familiar with intelligence maintained in June that GCHQ was " not looking at every piece of straw " but was sifting a "vast haystack of data " for what he called "needles " . 

Link found
95 [59, 68, 87, 95, 117]
Resist ! 
[alex442] Human Rights ? 
Haw haw ! 
Recall Parliament now ! 
Resist ! 
Freedom ? 

Link found
161 [6, 35, 51, 60, 74, 80, 121]
This disgraceful spying business has no end ! 
That operation , codenamed Tempora , has been running for 20 months . 
He added : " If you had the impression we are reading millions of email